# Spatial subsampling

This presents how spatial subsampling can be done using `py4dgeo`. 

As a first step, we import the `py4dgeo`package and specify our point cloud which we want to subsample where to store the output:

In [ ]:
import py4dgeo

infile = r"E:\test_data\240102_070000 - SINGLESCANS - 240102_070700.laz"
outfile = infile.replace(".laz", "_subsampled.laz")

When subsampling, one may want to keep additional dimensions. `py4dgeo` does not load any dimensions other than `X`,`Y`, or `Z` by default. Accordingly we have to define which dimensions we want to carry over from the original point cloud to the subsampled point cloud.

In [ ]:
dims = {"return_number": "return_number", "number_of_returns": "number_of_returns"}


# dim_dic = {"Reflectance":"Reflectance",
#            "number_of_returns":"number_of_returns",
#            "return_number":"return_number",
#            "red":"red",
#            "green":"green",
#            "blue":"blue",
#            "intensity":"intensity"}

epoch = py4dgeo.read_from_las(infile, additional_dimensions=dims)

The spatial subsampling will be done using `py4dgeo` by converting the Epoch into an Vapc object, which is allows voxel-based point cloud operations. Using this Vapc object we subsample to one point per voxel, accordingly the voxel_size parameter lets us control the spatial resolution. When subsampling one might want to do select which point to keep per voxel as this may have important implications for further processing. We offer the following options:
* "closest_to_centroid" -keeping the point closest to the centroid (R)
* "closest_to_voxel_centers" -keeping the point closest to the voxel center (R)
* "centroid" -keeping the centroid (S)
* "voxel_center" -keeping the voxel center (S)

Where real points (R) keep the attributes from the original point cloud, whereas syntetic points (S) get asigned the average of all points per voxel.

After reducing the point cloud to one point per voxel we save the output.

In [ ]:
# Let´s first mute vapc function trace and timeit for cleaner output
py4dgeo.enable_trace(False)
py4dgeo.enable_timeit(False)

voxel_size = 2
reduce_to_mode = "closest_to_centroid"  # other options are "closest_to_voxel_centers", "centroid", "voxel_center"
voxel_epoch = py4dgeo.Vapc(epoch, voxel_size=voxel_size)
reduced_vapc = voxel_epoch.reduce_to_feature(reduce_to_mode)
reduced_vapc.save_as_las(outfile=outfile)

As we are working it may be of interest to also save the voxels. The save_as_ply function allows us to do this by saving one voxel per point. It takes the edegelength from the voxel size. Features to be stored must be listed (in this example we write all features), and the "mode" allows us to define the center of each voxel. Here the same options as in the reduce to method are available: "closest_to_centroid", "closest_to_voxel_centers", "centroid", and "voxel_center".

In [ ]:
reduced_vapc.save_as_ply(
    outfile=outfile.replace(".laz", ".ply"),
    features=reduced_vapc.out.keys(),
    mode=reduce_to_mode,
)